# Tokens and analysis

This demo is based on 123844 tokens with 205005 analysis.

We have a file of tokens with analyses where every line has json encoded form
```
token, [(lemma, root, root_tokens, ending, clitic, partofspeech, form), ...]
```
For example:
```
["Ühtedel", [["üks", "üks", ["üks"], "del", "", "N", "pl ad"], ["üks", "üks", ["üks"], "del", "", "P", "pl ad"]]]
```

## Create a dict analysis_examples
```
{(cap, dash, eq, ending, clitic, partofspeech, form): token, ...}
```
where 

        cap == True, if token starts with a capital letter,
        cap == False, if token starts with a lower case letter,
        cap == None, if token does not start with a letter,
        dash == True, if '-' in the root,
        dast == False if '-' not in the root,
        eq == True, if '=' in the root,
        eq == False if '=' not in the root.

In [1]:
from collections import Counter
from tokens_analysis import yield_tokens_analysis
file='../corpora/tokens_analysis.json'

analysis_examples = {}
analysis_counter = Counter()
for token, analysis in yield_tokens_analysis(file):
    cap = token[0].isupper()
    for a in analysis:
        key = (cap, 
               '-' in a['root'], 
               '=' in a['root'], 
               a['ending'], 
               a['clitic'], 
               a['partofspeech'], 
               a['form'])
        analysis_examples[key] = token
        analysis_counter[key] += 1

print('Ten most common analysis.\n')
for a, n in analysis_counter.most_common(10):
    print('{:<45} {:<20} {:>10}'.format(str(a), analysis_examples[a], n))

Ten most common analysis.

(True, False, False, '0', '', 'H', 'sg n')    Προϊόντα                  18159
(True, False, False, '0', '', 'H', 'sg g')    Εcu                       13890
(False, False, False, '0', '', 'S', 'sg n')   τόνο                      12430
(False, False, False, '0', '', 'S', 'sg g')   žürii                     12360
(False, False, False, '0', '', 'S', 'sg p')   žiletitera                 5051
(False, False, False, 'd', '', 'S', 'pl n')   žüriid                     4680
(False, False, False, '0', '', 'S', 'adt')    želatiini                  4055
(True, False, False, '0', '', 'S', 'sg n')    Žürii                      3654
(False, False, False, 't', '', 'S', 'sg p')   žiirot                     3431
(True, False, False, '0', '', 'S', 'sg g')    Žürii                      3353


## Text of tokens that have pairwise different analysis

Maybe we can use this text for testing syntax_preprocessing efficiently.

In [2]:
text = ' '.join(set(analysis_examples.values()))
text[:1000]

'Vastavat peaksidki Paljudelt Pikendamisel Opel-taksosse üritasingi üüripinnalt Sedagi Mendes Parempoolsetega väiksemast kolmandat-neljandat Põhja-Euroopasse õlule õppejõududelt üksikasju tuhandetesse ŠANK Venemaagi üle-elusuurust üldhariduslikus VAZ-2106 öeldud ülema provokatiivseltki V-liine Väiksemad UFO-mänguasju Samu üldisemas vahevalit-sust võinuksite vanematega al-Qaidaga teis Tänaseni Ähvardavad Üheksandat Kustuga ütlete Teistega Võimalikele Soome-ugri õrnemaid selletagi ühelgi Üllatusena ühele-kahele Pooltel samadelt üüritakse ühessegi Ülim bi-le ümbriseta ütlustes teisteski Tänavail toodanguvõimelisemateks õpetatav värskeima Mõnelegi neilgi nDile Varema NATOt Õppe-kasvatustööst üürib viitest ütlesite Üldhariduslike Kellelegi õitsvaim Kaalukaimad ütelda Piisanuks On-on ülem- Teie ühistulisele tõu-su Εcu eeskujudetagi WEU-d Sadade Teistest ühtegi ülevaadeteks teid üksikasjalisemalt ärgates On Talle -Bistrita Üürime Käinudki võõrsõnastikega Üksikasjalisemat žsamaaliidul Truuduse

## Pronouns in corpora

Find all tokens that are marked as pronouns (partofspeech P). Group the pronouns by regexes from \_pronConversions. The class 'no_search_match' is for pronouns that don't match any regex.

In [3]:
_pronConversions = ( ("emb\+.*",             ("det",)),
                     ("enda\+.*",            ("pos", "refl")),
                     ("enese\+.*",           ("pos", "refl")),
                     ("eikeegi.*",           ("indef",)),
                     ("eimiski.*",           ("indef",)),
                     ("emb-kumb.*",          ("det",)),
                     ("esimene.*",           ("dem",)),
                     ("iga\+.*",             ("det",)),
                     ("iga_sugune.*",        ("indef",)),
                     ("iga_.ks\+.*",         ("det",)),
                     ("ise\+.*",             ("pos", "det", "refl")),
                     ("ise_enese.*",         ("refl",)),
                     ("ise_sugune.*",        ("dem",)),
                     ("keegi.*",             ("indef",)),
                     ("kes.*",               ("inter rel",)),
                     ("kumb\+.*",            ("rel",)),
                     ("kumbki.*",            ("det",)),
                     ("kõik.*",              ("det",)),
                     ("k.ik.*",              ("det",)),
                     ("meie_sugune.*",       ("dem",)),
                     ("meie_taoline.*",      ("dem",)),
                     ("mihuke\+.*",          ("inter rel",)),
                     ("mihukene\+.*",        ("inter rel",)),
                     ("mille_taoline.*",     ("dem",)),
                     ("milli=?ne.*",         ("rel",)),
                     ("mina\+.*",            ("pers ps1",)),
                     (" ma\+.*",             ("pers ps1",)),
                     ("mina=?kene\+.*",      ("dem",)),
                     ("mina=?ke\+.*",        ("dem",)),
                     ("mingi\+.*",           ("indef",)),
                     ("mingi_sugune.*",      ("indef",)),
                     ("minu_sugune.*",       ("dem",)),
                     ("minu_taoline.*",      ("dem",)),
                     ("miski.*",             ("indef",)),
                     ("mis\+.*",             ("inter rel",)),
                     ("mis_sugune.*",        ("inter rel",)),
                     ("miski\+.*",           ("inter rel",)),
                     ("miski_sugune.*",      ("inter rel",)),
                     ("misu=?ke(ne)?\+.*",   ("dem",)),
                     ("mitme_sugune.*",      ("indef",)),
                     ("mitme_taoline.*",     ("indef",)),
                     ("mitmendik\+.*",       ("inter rel",)),
                     ("mitmes\+.*",          ("inter rel", "indef")),
                     ("mi=?tu.*",            ("indef",)),
                     ("miuke(ne)?\+.*",      ("inter rel",)),
                     ("muist\+.*",           ("indef",)),
                     ("muu.*",               ("indef",)),
                     ("m.lema.*",            ("det",)),
                     ("m.ne_sugune\+.*",     ("indef",)),
                     ("m.ni\+.*",            ("indef",)),
                     ("m.ningane\+.*",       ("indef",)),
                     ("m.ningas.*",          ("indef",)),
                     ("m.herdune\+.*",       ("indef", "rel")),
                     ("määntne\+.*",         ("dem",)),
                     ("na_sugune.*",         ("dem",)),
                     ("nende_sugune.*",      ("dem",)),
                     ("nende_taoline.*",     ("dem",)),
                     ("nihuke(ne)?\+.*",     ("dem",)),
                     ("nii_mi=?tu\+.*",      ("indef", "inter rel")),
                     ("nii_sugune.*",        ("dem",)),
                     ("niisama_sugune.*",    ("dem",)),
                     ("nii?su=?ke(ne)?\+.*", ("dem",)),
                     ("niuke(ne)?\+.*",      ("dem",)),
                     ("oma\+.*",             ("pos", "det", "refl")),
                     ("oma_enese\+.*",       ("pos",)),
                     ("oma_sugune\+.*",      ("dem",)),
                     ("oma_taoline\+.*",     ("dem",)),
                     ("palju.*",             ("indef",)),
                     ("sama\+.*",            ("dem",)),
                     ("sama_sugune\+.*",     ("dem",)),
                     ("sama_taoline\+.*",    ("dem",)),
                     ("samune\+.*",          ("dem",)),
                     ("see\+.*",             ("dem",)),
                     ("see_sama\+.*",        ("dem",)),
                     ("see_sam[au]ne\+.*",   ("dem",)),
                     ("see_sinane\+.*",      ("dem",)),
                     ("see_sugune\+.*",      ("dem",)),
                     ("selle_taoline\+.*",   ("dem",)),
                     ("selli=?ne\+.*",       ("dem",)),
                     ("setu\+.*",            ("indef",)),
                     ("setmes\+.*",          ("indef",)),
                     ("sihuke\+.*",          ("dem",)),
                     ("sina\+.*",            ("pers ps2",)),
                     (" sa\+.*",             ("pers ps2",)),
                     ("sinu_sugune\+.*",     ("dem",)),
                     ("sinu_taoline\+.*",    ("dem",)),
                     ("siuke(ne)?\+.*",      ("dem",)),
                     ("säherdune\+.*",       ("dem",)),
                     ("s.herdune\+.*",       ("dem",)),
                     ("säärane\+.*",         ("dem",)),
                     ("s..rane\+.*",         ("dem",)),
                     ("taoline\+.*",         ("dem",)),
                     ("teie_sugune\+.*",     ("dem",)),
                     ("teie_taoline\+.*",    ("dem",)),
                     ("teine\+.*",           ("dem",)),
                     ("teine_teise\+.*",     ("rec",)),
                     ("teist?_sugune\+.*",   ("dem",)),
                     ("tema\+.*",            ("pers ps3",)),
                     (" ta\+.*",             ("pers ps3",)),
                     ("temake(ne)?\+.*",     ("pers ps3",)),
                     ("tema_sugune\+.*",     ("dem",)),
                     ("tema_taoline\+.*",    ("dem",)),
                     ("too\+.*",             ("dem",)),
                     ("too_sama\+.*",        ("dem",)),
                     ("üks.*",               ("dem", "indef")),
                     (".ks.*",               ("dem", "indef")),
                     ("ükski.*",             ("dem", "indef")),
                     (".kski.*",             ("dem", "indef")),
                     ("üks_teise.*",         ("rec", "indef")),
                     (".ks_teise.*",         ("rec",))
)

import regex

search = regex.search
match = regex.match
pronoun_examples = {}
for a, b in _pronConversions:
    pronoun_examples[a] = []
no_search_match_examples = []

for token, analysis in yield_tokens_analysis(file):
    for a in analysis:
        if a['partofspeech'] != 'P':
            continue
        root_ec = ''.join((a['root'], '+', a['ending'], a['clitic']))
        search_match = False
        for pattern, l in pronoun_examples.items():
            if search(pattern, root_ec):
                # kas search või match? "enese" vs "iseenese"
                # kas token asemel võiks olla hoopis token.lower() või root või lemma (ja l oleks siis list asemel set)?
                l.append(token)
                search_match = True
        if not search_match:
            no_search_match_examples.append(token)
pronoun_examples['no_search_match'] = no_search_match_examples
for pattern, examples in pronoun_examples.items():
    print(pattern)
    print(examples)

mihuke\+.*
['Mihuke', 'Mihukeselt']
mina=?kene\+.*
[]
oma_enese\+.*
['Omaenese', 'omaenda', 'omaenese']
oma_sugune\+.*
['omasuguseid', 'omasuguste', 'omasugustega']
 ma\+.*
[]
enda\+.*
['täiendada-muuta']
mihukene\+.*
['Mihukeselt']
nii?su=?ke(ne)?\+.*
[]
nende_sugune.*
[]
mis_sugune.*
['Missugune', 'Missuguse', 'Missugused', 'Missuguses', 'Missugust', 'missugune', 'missuguse', 'missugused', 'missuguseid', 'missuguseks', 'missugusele', 'missuguses', 'missugust', 'missuguste', 'missugustel']
too\+.*
['Nood', 'Tol', 'Tolle', 'Tolleks', 'Tollesse', 'Tollest', 'Too', 'nois', 'nonde', 'nood', 'toda', 'tol', 'tolle', 'tolleks', 'tollel', 'tollele', 'tolles', 'tollest', 'too']
teine_teise\+.*
['Teineteise', 'Teineteise', 'Teineteisele', 'teineteise', 'teineteise', 'teineteisega', 'teineteisel', 'teineteisele', 'teineteisest', 'teineteist']
emb-kumb.*
['emb-kumb', 'emba-kumba', 'emba-kumba']
teist?_sugune\+.*
['Teistsugused', 'Teistsuguste', 'teistsugune', 'teistsuguse', 'teistsugused', 'teist

## Morph analysis of tokens *vs* tmorftrtabel.txt

In [4]:
pos_form_pairs = {}
from estnltk.rewriting.syntax_preprocessing.syntax_preprocessing import MorphToSyntaxMorphRewriter
tmorph_file = '../estnltk/estnltk/rewriting/syntax_preprocessing/rules_files/tmorftrtabel.txt'
pos_form_pairs['in tmorftrtabel.txt'] = set(MorphToSyntaxMorphRewriter.load_fs_mrf_to_syntax_mrf_translation_rules(tmorph_file))

pos_form_pairs['in corpus'] = set()
for a in analysis_examples:
    pos_form_pairs['in corpus'].add((a[5], a[6]))
    
print('(pos, form) pairs in tmorftrtabel.txt, but not in corpus:')
print(pos_form_pairs['in tmorftrtabel.txt'] - pos_form_pairs['in corpus'])
print('\n(pos, form) pairs in corpus, but not in tmorftrtabel.txt:')
not_in_tmorftrtabel = pos_form_pairs['in corpus'] - pos_form_pairs['in tmorftrtabel.txt']
for pos_form in not_in_tmorftrtabel:
    print('{:<20}'.format(str(pos_form)), end='')
    for a in analysis_examples:
        if pos_form == (a[5], a[6]):
            print(analysis_examples[a], end=', ')
    print()

(pos, form) pairs in tmorftrtabel.txt, but not in corpus:
{('O', 'pl ter'), ('P', ''), ('Y', 'pl ter'), ('U', 'pl abl'), ('U', 'pl ab'), ('U', 'pl ill'), ('C', 'pl ter'), ('Y', 'pl abl'), ('Y', 'pl ill'), ('C', 'sg ter'), ('U', 'sg ter'), ('N', 'pl ab'), ('U', 'sg kom'), ('U', 'pl kom'), ('N', 'pl tr'), ('O', 'sg ter'), ('N', 'sg ab'), ('U', 'sg ab'), ('U', 'pl es'), ('Y', 'pl tr'), ('V', 'tuvat'), ('N', 'pl ter'), ('Y', 'pl es'), ('Y', 'pl all'), ('O', 'sg ab'), ('V', 'nuvat'), ('Y', 'sg n'), ('U', 'pl ter')}

(pos, form) pairs in corpus, but not in tmorftrtabel.txt:
('V', 'neg me')     ärme, Ärme, 
('H', '')           Virisemata, Ülemöödunud, 


## Roots that are in corpus but not in abileksikon06.utf.lx

In [5]:
# find all pairs (root, partofspeech) from abileksikon
abileksikon_file = '../estnltk/estnltk/rewriting/syntax_preprocessing/rules_files/abileksikon06utf.lx'

root_pos = set()
with open(abileksikon_file, mode='r') as f:
    while True:
        root = next(f, None)
        if root is None:
            break
        root = root.rstrip('\n')
        rule = next(f).rstrip('\n')
        if ('_K_' in rule):
            root_pos.add((root, 'K'))
        if ('_V_' in rule):
            root_pos.add((root, 'V'))
        if ('_Y_' in rule):
            root_pos.add((root, 'Y'))
        if ('_Z_' in rule):
            root_pos.add((root, 'Z'))

not_in_abileksikon = {'K': set(), 'V': set(), 'Y': set(), 'Z': set()}

file='../corpora/tokens_analysis.json'
for token, analysis in yield_tokens_analysis(file):
    for a in analysis:
        if a['partofspeech'] in {'K', 'V'}:
            if (a['root'], a['partofspeech']) not in root_pos:
                not_in_abileksikon[a['partofspeech']].add(a['root'])
        elif a['partofspeech'] in {'Y', 'Z'}:
            if len(a['root']) > 1:
                continue
            if (a['root'], a['partofspeech']) not in root_pos:
                not_in_abileksikon[a['partofspeech']].add(a['root'])

# Y ja Z jaoks me abileksikoni syntax_preprocessingu juures ei kasuta,
# piisab PunctuationTypeRewriter-st.
print('Y (max one character)')
print(not_in_abileksikon['Z'])
print('Z (max one character)')
print(not_in_abileksikon['Z'])
print('K')
print(not_in_abileksikon['K'])
print('V')

print(not_in_abileksikon['V'])

Y (max one character)
{'<', '', '‹', '>', '=', '’', '›', '*', ']', '+', '/', '['}
Z (max one character)
{'<', '', '‹', '>', '=', '’', '›', '*', ']', '+', '/', '['}
K
{'jõulu-ajal', 'arust', 'alates', 'maa-rahu-eest', 'vastavalt', 'peast', 'kohaselt', 'ilma+st-ilma'}
V
{'friigi', 'nai', 'homofoopi', 'vooderda', 'aktsepti', 'liiklusreidi', 'liigenda', 'jubu', 'madalda', 'kuu_päevasta', 'tsŠiili', 'toodri', 'aktsepteeri', 'paari', 'nosi', 'ummista', 'noteeri', 'rehkenda', 'või', 'ees_õigusta', 'threadi', 'vihtle', 'lossi', 'kogele', 'kukku', 'diagnoos+e-ürita', 'rüüpa', 'pommi', 'diplomeeri', 'ümruskonda', 'kappi', 'aja_kohasta', 'salasta', 'loksuta', 'õhka', 'avita', 'vuhki', 'lõõtsu', 'praugu', 'stabiliseeri', 'poskijetta', 'aeglusta', 'pinguta', 'kibele', 'peesita', 'laosta', 'sõtku', 'testosterooni', 'samasta', 'siir', 'kontrasti', 'panusta', 'kaas_ajasta', 'viksi', 'jututa', 'lehvita', 'prantsata', 'udja', 'postiivne', 'nori', 'punkti', 'lahjenda', 'karmista', 'lipsata', 'soiku', 'sa

## Everything that comes afer '=' in root

Group the results by part of speech.

In [6]:
from collections import defaultdict
root_eq_ends = defaultdict(set)
file = '../corpora/tokens_analysis.json'
for token, analysis in yield_tokens_analysis(file):
    for a in analysis:
        if '=' in a['root']:
            root_split = a['root'].split('=')
            if len(root_split) > 2:
                print('more than one "=" in root:b', a['root'])
            root_eq_ends[a['partofspeech']].add(root_split[-1])
for key, values in root_eq_ends.items():
    print(key)
    print(values)

more than one "=" in root:b mõel=nu+d-tei=nud
more than one "=" in root:b mõel=nu+d-tei=nud
more than one "=" in root:b mõel=nu+d-tei=nud
more than one "=" in root:b sorki=mis-nuusuta=mis_meetod
D
{'likult', 'malt', 'likumalt', 'numalt', 'nult', 'valt', 'tamatult', 'ti', 'istlikult', 'lt', 'matult', 'lisemalt', 'mini', 'laslikult', 'istlikumalt', 'vamalt', 'konniti', 'tult', 'liselt', 'tavamalt'}
C
{'tum', 'matum', 'tavam', 'vam', 'istlikum', 'm', 'lisem', 'likum', 'num'}
Z
{''}
U
{'im', 'tavaim', 'tuim', 'duim', 'vaim', 'lisim', 'nuim'}
O
{'s'}
H
{'ist', 'vam', 'istlik', 'mis_keskus', 'v', 'nu', 'mis_klubi', 'tav', 'ja', 'kus', 'tuim', 'lisem', 'mine', 'ja+te_koda', 'lik', 'tu', 'ke', 'kene', 'likkus', 'line', 'mis_taotlus', 'likum', 'tuslik', 'im', 'laslik', 'ja+te_ühistu', 'm', 'vaim', 'mata', 'tus', 'sus'}
S
{'mis_protokoll', 'tu+d-armastatu', 'mis_vastane', 'mis_nähe', 'mis_meetod', 'mis_kinnitus', 'mis_keskus', 'mis_pjedestaal', 'mis_klubi', 'ja+te_vahetus', 'mis_luba', 'mis_kee'

## All tokens that have part of speech "V" and form "neg"

In [7]:
file = '../corpora/tokens_analysis.json'
aux_neg_tokens = []
for token, analysis in yield_tokens_analysis(file):
    for a in analysis:
        if a['form'] == 'neg' and a['partofspeech'] == 'V':
            aux_neg_tokens.append(token)
            break
aux_neg_tokens

['EI', 'Ei', 'ei']